In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from db_queries import get_location_metadata

import warnings
warnings.simplefilter('ignore')


In [ ]:
loc_df = get_location_metadata(location_set_id=111, location_set_version_id=677)
loc_df = loc_df.loc[(loc_df['path_to_top_parent'].str.startswith('102,')) & (loc_df['most_detailed'] == 1), 
                    ['location_id', 'location_name']]

est_dfs = []
for i in range(7):
    est_df = pd.read_csv(f'/ihme/covid-19/deaths/dev/2020_05_23_cfrmod_sub{i+8}/model_results.csv')
    est_df['date'] = pd.to_datetime(est_df['date'])
    est_df['Predicted deaths'] = np.nan
    est_df['Predicted deaths'][1:] = np.diff(est_df[[f'draw_{d}' for d in range(1000)]], axis=0).mean(axis=1)
    est_df = est_df.loc[est_df['date'] > est_df.groupby('location_id')['date'].transform('max') - pd.Timedelta(days=8)]
    est_dfs.append(est_df[['location_id', 'date', 'Predicted deaths']])

data_df = pd.read_csv('/ihme/covid-19/deaths/dev/2020_05_23_cfrmod/model_data.csv')
data_df['date'] = pd.to_datetime(data_df['Date'])
data_df['Observed deaths'] = np.nan
data_df['Observed deaths'][1:] = np.diff(data_df['Death rate'] * data_df['population'])
data_df = data_df[['location_id', 'date', 'Observed deaths']]

df = data_df.merge(est_df)
df = loc_df.merge(df)
df = df.loc[~df['Observed deaths'].isnull()]


In [ ]:
df = df.groupby('location_name', as_index=False)['Observed deaths', 'Predicted deaths'].sum()

In [ ]:
plt.scatter(np.log(df['Observed deaths']),
            np.log(df['Predicted deaths']))
plt.plot(np.log((1, 1400)), np.log((1, 1400)), color='red')
plt.yticks(np.arange(0, 8), np.exp(np.arange(0, 8)).astype(int))
plt.xticks(np.arange(0, 8), np.exp(np.arange(0, 8)).astype(int))
plt.xlabel('Observed')
plt.ylabel('Predicted')
plt.show()